In [ ]:
# The purpose of this file is to create datasets of tweets about each of the teams participating in VCT Champions 2021.
# The file uses Tweepy to pull tweets using a given query and stores them as CSVs using pandas.
# A sentiment analysis is conducted on all tweets, using the textblob library

In [ ]:
import os
import tweepy
import pandas as pd
import re
import webbrowser
from textblob import TextBlob
from wordcloud import WordCloud

In [ ]:
# Authentication

consumer_key = "" # Omitted for security reasons
consumer_secret = "" # Omitted for security reasons
callback_uri = 'oob' # https://cfe.sh/twitter/callback
auth = tweepy.OAuthHandler(consumer_key, consumer_secret, callback_uri)
redirect_url = auth.get_authorization_url()
webbrowser.open(redirect_url)
user_pin_input = input("What's the pin value?")
auth.get_access_token(user_pin_input) # auth.access_token & auth.access_token_secret

In [ ]:
# Verifying that the api is working, prints my username (aliasidd04)

api = tweepy.API(auth, wait_on_rate_limit = True)
me = api.me()
print(me.screen_name)

In [ ]:
def extract_list_as_df(timeline_list, team, playing_today):
    
    # Takes in a list of tweets, generated through tweepy, and converts the list into a pandas dataframe
    
    columns = set()
    allowed_types = [str, int]
    tweets_data = [] # List that will contain the data for the dataframe
    
    for tweet in timeline_list: # For each tweet in the list of tweets pulled from tweepy

        status_obj_url = f"https://twitter.com/{tweet['screen_name']}/status/{tweet['id']}" # Creates a link for tweet, to be added to the dataframe
        
        single_tweet_data = {"user": tweet['screen_name'], # Username of the tweeter
                             "id": tweet['id'], # Unique ID of tweets
                             "text": tweet['text'], # Text in the tweet
                             "url": status_obj_url, # URL generated above
                             "datetime": tweet['date'], # Date and time of tweet is UTC
                             "playing_today": playing_today} # Custom field I manually updated each day for each team, to help label which teams were playing on what days
        
        tweets_data.append(single_tweet_data)
        
    header_cols = list(columns) # Creating the column names of the dataframe
    header_cols.extend(("user","id", "text","url","datetime", "playing_today"))

    df = pd.DataFrame(tweets_data, columns = header_cols)    
    return df

In [ ]:
def cleanText(text):
    
    # Takes in the list of tweet text and removes any unecessary symbols, numbers, or letters which may interfere with the sentiment analysis
    
    text = re.sub(r'(@[A-Za-z]+)|([:_]+)', '', text) # remove @ mentions
    text = re.sub(r'#', '', text) # remove # symbol
    text = re.sub(r'RT[\s]+', '', text) # remove retweets
    text = re.sub(r'https?:\/\/\S+', '', text) # remove hyperlinks
    text = re.sub(r'https\S+', '', text) # remove hyperlinks
    
    return text

In [ ]:
def getResults(query, day):
    results = []

    for i, status in enumerate(tweepy.Cursor(api.search, q=query, since=day,lang="en").items(10000)):
        tweet = {}
        tweet['user_id']  = str(status.user.id)
        tweet['id'] = str(status.id)
        tweet['screen_name'] = status.user.screen_name
        tweet['text'] = status.text
        tweet['date'] = status.created_at
        tweet['date'] = tweet['date'].strftime('%Y-%m-%d, %H:%M:%S')
        results.append(tweet.copy())
        
    return results

In [ ]:
def getSubjectivity(text):
   
    # Function to get the subjectivity of a tweet (between 0 and 1, 1 being more opinionated and 0 being more factual
    
    return TextBlob(text).sentiment.subjectivity

def getPolarity(text):
    
    # Create function to get the polarity (how positive or negative the function is)
    
    return TextBlob(text).sentiment.polarity

In [ ]:
def getAnalysis(score):
    
    # Function to compute the negative, neutral and positive analysis, based on a tweets polarity
    
    if score < 0:
        return 'Negative'
    elif score == 0:
        return 'Neutral'
    else:
        return 'Positive'

In [ ]:
team = 'acend' # Team for which the tweets are being pulled
day = '2021-12-11' # Day of the tournament for which the tweets are being pulled
playing_today = True # False if the team played on that day, True if they did
path = f'C:\\Users\\Ali\\Desktop\\Coding\\Data Science Practice\\Twitter\\CSV Files\\{day}' 
query = f"{team} #VCT OR {team} #VALORANTChampions -filter:retweets -from:valesports_na -from:ValorantEsports" # Query that is sent through tweepy

results = getResults(query,day) # Pulls
df = extract_list_as_df(results, team, playing_today)

pd.set_option('max_colwidth', 400)
df['text']= df['text'].apply(cleanText)

df['text'] = df['text'].str.replace(r'[^\x00-\x7F]+', '', regex=True)

df['Subjectivity'] = df['text'].apply(getSubjectivity)
df['Polarity'] = df['text'].apply(getPolarity)

df['Analysis'] = df['Polarity'].apply(getAnalysis)

df

In [ ]:
df.to_csv(os.path.join(path,f'{team} ({day}).csv'))